In [4]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import yaml
import xopen
import json
import warnings
warnings.filterwarnings("ignore")
from os.path import dirname
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer
from data import infer_preprocess
from model import Classifier, BERTDiseaseClassifier
from utils import default_symps
from nltk.tokenize import sent_tokenize
import blingfire
import spacy

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


ModuleNotFoundError: No module named 'torch'

In [3]:
def cut_sentences(text, tokenizer, nlp):
    if tokenizer == 'blingfire':
        sents = blingfire.text_to_sentences(text.strip()).split("\n")
    if tokenizer == 'nltk':
        sents = sent_tokenize(text.strip())
    if tokenizer == 'spacysm':
        doc = nlp(text)
        sents = [sent.text.strip() for sent in doc.sents]
    if tokenizer == 'spacylg':
        doc = nlp(text)
        sents = [sent.text.strip() for sent in doc.sents]
    if tokenizer == 'spacytrf':
        doc = nlp(text)
        sents = [sent.text.strip() for sent in doc.sents]
    return sents

In [34]:
datastore = []
# options : blingfire, nltk, spacysm, spacylg, spacytrf
senttokenizer = 'spacysm'
# set spacy tokenizer
if senttokenizer == 'spacysm':
    nlp = spacy.load("en_core_web_sm")
if senttokenizer == 'spacylg':
    nlp = spacy.load("en_core_web_lg")
if senttokenizer == 'spacytrf':
    nlp = spacy.load("en_core_web_trf")    

symps = ["Anxious_Mood","Autonomic_symptoms","Cardiovascular_symptoms","Catatonic_behavior","Decreased_energy_tiredness_fatigue","Depressed_Mood","Gastrointestinal_symptoms","Genitourinary_symptoms","Hyperactivity_agitation","Impulsivity","Inattention","Indecisiveness","Respiratory_symptoms","Suicidal_ideas","Worthlessness_and_guilty","avoidance_of_stimuli","compensatory_behaviors_to_prevent_weight_gain","compulsions","diminished_emotional_expression","do_things_easily_get_painful_consequences","drastical_shift_in_mood_and_energy","fear_about_social_situations","fear_of_gaining_weight","fears_of_being_negatively_evaluated","flight_of_ideas","intrusion_symptoms","loss_of_interest_or_motivation","more_talktive","obsession","panic_fear","pessimism","poor_memory","sleep_disturbance","somatic_muscle","somatic_symptoms_others","somatic_symptoms_sensory","weight_and_appetite_change","Anger_Irritability"]

if __name__ == "__main__":
    batch_size = 64
    input_dir = "../../../data/postdatalines.json"
    ckpt_dir = "lightning_logs/version_0/checkpoints/epoch=0-step=720.ckpt"
    hparams_dir = os.path.join(dirname(dirname(ckpt_dir)), 'hparams.yaml')
    hparams = yaml.load(open(hparams_dir),Loader=yaml.Loader)
    max_len = hparams["max_len"]
    tokenizer = AutoTokenizer.from_pretrained(hparams["model_type"])
    clf = Classifier.load_from_checkpoint(ckpt_dir, symps=default_symps)
    clf.eval()
    clf.cuda()

    allPostSentences = []
    
    with xopen.xopen(input_dir) as fi:
        for i, line in tqdm(enumerate(fi)):
            record = json.loads(line)
            
            user_sents = []
            sent_bounds = [0]
            curr_sid = 0
            
            post_data = {
            "gender": record["gender"],
            "label": record["label"],
            "sentences": []
            }   

            if record['text'] == None:
                break
            else:
                for post in record["text"]:
                    sents = cut_sentences(post, senttokenizer, nlp)
                    curr_sid += len(sents)
                    sent_bounds.append(curr_sid)
                    user_sents.extend(sents)

                
                all_probs = []
                for i in range(0, len(user_sents), batch_size):
                    curr_texts = user_sents[i:i+batch_size]
                    processed_batch = infer_preprocess(curr_texts, tokenizer, max_len)
                    for k, v in processed_batch.items():
                        processed_batch[k] = v.cuda()
                    with torch.no_grad():
                        feats, logits = clf.feat_extract_avg(processed_batch)
                        probs = logits.sigmoid().detach().cpu().numpy()
                    all_probs.extend(probs)
                
                for sent, probs in zip(user_sents, all_probs):
                    sent_data = {"sentence": sent, "probabilities": dict(zip(symps, probs))}
                    post_data['sentences'].append(sent_data)
                
                allPostSentences.append(post_data)

Some weights of BertModel were not initialized from the model checkpoint at mental/mental-bert-base-uncased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


"accumulation":       1
"bal_sample":         True
"bs":                 64
"control_ratio":      0.5
"exp_name":           mbert_label_enhance_bal_sample_050_666
"gradient_clip_val":  0.1
"input_dir":          ../data/symp_data_w_control
"loss_mask":          True
"loss_type":          bce
"loss_weighting":     mean
"lr":                 0.0003
"max_len":            64
"model_type":         mental/mental-bert-base-uncased
"patience":           4
"pos_weight_setting": default
"seed":               666
"threshold":          0.5
"uncertain":          exclude
"write_result_dir":   ./lightning_logs/bal_sample_records.json


797it [03:15,  4.07it/s]


In [5]:
import pandas as pd

In [6]:
try:
 df = pd.read_json('../../../data/annotationData/sentencedata.json',orient='records')
except:
 df = pd.DataFrame(allPostSentences)
 df.to_json('../../../data/annotationData/sentencedata.json', orient='records')

female = df.loc[(df['label'] == 'depression') & (df['gender'] == 'f')]
male = df.loc[(df['label'] == 'depression') & (df['gender'] == 'm')]

femaleAnnotation = female.sample(n=20, random_state=99)
maleAnnotation = male.sample(n=20, random_state=99)

femaleAnnotation.to_json('../../../data/annotationData/femaledata.json',orient='records')
maleAnnotation.to_json('../../../data/annotationData/maledata.json',orient='records')


In [133]:
gender = 'male'
df = pd.read_json(f'../../../data/annotationData/{gender}data.json', orient='records')
symps = ["Anxious_Mood","Autonomic_symptoms","Cardiovascular_symptoms","Catatonic_behavior","Decreased_energy_tiredness_fatigue","Depressed_Mood","Gastrointestinal_symptoms","Genitourinary_symptoms","Hyperactivity_agitation","Impulsivity","Inattention","Indecisiveness","Respiratory_symptoms","Suicidal_ideas","Worthlessness_and_guilty","avoidance_of_stimuli","compensatory_behaviors_to_prevent_weight_gain","compulsions","diminished_emotional_expression","do_things_easily_get_painful_consequences","drastical_shift_in_mood_and_energy","fear_about_social_situations","fear_of_gaining_weight","fears_of_being_negatively_evaluated","flight_of_ideas","intrusion_symptoms","loss_of_interest_or_motivation","more_talktive","obsession","panic_fear","pessimism","poor_memory","sleep_disturbance","somatic_muscle","somatic_symptoms_others","somatic_symptoms_sensory","weight_and_appetite_change","Anger_Irritability"]

relevant_sents = []
postid = 0
sentid = 0

for post in df['sentences']:
    postid += 1
    for sent in post:
        sentid += 1
        meet_threshold = []
        if len(sent['sentence'].split()) >= 10:
            meet_threshold = [{'id':[postid,sentid],'sentence':sent['sentence']}]
            for symptom in symps:
                if sent['probabilities'][symptom] >= 0.3:
                    meet_threshold.append({symptom:sent['probabilities'][symptom]})
                    if sent['probabilities'][symptom] >= 0.5:
                        meet_threshold.append('1')
                    else:
                        meet_threshold.append('0')
            if len(meet_threshold) >= 4:
                relevant_sents.append(meet_threshold)
        else:
            continue


print(len(relevant_sents))

df = pd.DataFrame(relevant_sents)
# df.to_json(f'../../../data/annotationData/{gender}relevantdata.json', orient='records')
df


116


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,"{'id': [1, 4], 'sentence': 'and then some depr...",{'Anxious_Mood': 0.8263251185},1,{'Depressed_Mood': 0.7565150261},1,{'flight_of_ideas': 0.5915693641},1,{'intrusion_symptoms': 0.7555935383},1,{'obsession': 0.8809010983000001},1,{'Anger_Irritability': 0.3181036711},0,None,None
1,"{'id': [1, 8], 'sentence': 'and I could tell t...",{'Depressed_Mood': 0.310393244},0,{'Worthlessness_and_guilty': 0.9204751849},1,None,None,None,None,None,None,None,None,None,None
2,"{'id': [1, 11], 'sentence': 'and I just Loop t...",{'Anxious_Mood': 0.47901782390000003},0,{'pessimism': 0.804972291},1,None,None,None,None,None,None,None,None,None,None
3,"{'id': [1, 21], 'sentence': 'I never thought a...",{'Depressed_Mood': 0.9508749247},1,{'do_things_easily_get_painful_consequences': ...,1,{'drastical_shift_in_mood_and_energy': 0.33102...,0,None,None,None,None,None,None,None,None
4,"{'id': [1, 23], 'sentence': 'and so I spent th...",{'Depressed_Mood': 0.6639720798000001},1,{'Worthlessness_and_guilty': 0.8920667768},1,{'pessimism': 0.6086922288000001},1,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,"{'id': [19, 855], 'sentence': 'and i would say...",{'Depressed_Mood': 0.9589453936000001},1,{'diminished_emotional_expression': 0.61467522...,1,{'flight_of_ideas': 0.5183463097000001},1,{'intrusion_symptoms': 0.37277352810000003},0,None,None,None,None,None,None
112,"{'id': [19, 856], 'sentence': 'um and you know...",{'flight_of_ideas': 0.7926652431000001},1,{'pessimism': 0.3433488905},0,None,None,None,None,None,None,None,None,None,None
113,"{'id': [19, 859], 'sentence': 'why and i knew ...",{'Depressed_Mood': 0.9687442183},1,{'Worthlessness_and_guilty': 0.5315486789},1,None,None,None,None,None,None,None,None,None,None
114,"{'id': [20, 872], 'sentence': 'and I accept th...",{'fear_of_gaining_weight': 0.9143326283000001},1,{'weight_and_appetite_change': 0.9465436935},1,None,None,None,None,None,None,None,None,None,None


In [132]:
gender = 'female'
df = pd.read_json(f'../../../data/annotationData/{gender}data.json', orient='records')
symps = ["Anxious_Mood","Autonomic_symptoms","Cardiovascular_symptoms","Catatonic_behavior","Decreased_energy_tiredness_fatigue","Depressed_Mood","Gastrointestinal_symptoms","Genitourinary_symptoms","Hyperactivity_agitation","Impulsivity","Inattention","Indecisiveness","Respiratory_symptoms","Suicidal_ideas","Worthlessness_and_guilty","avoidance_of_stimuli","compensatory_behaviors_to_prevent_weight_gain","compulsions","diminished_emotional_expression","do_things_easily_get_painful_consequences","drastical_shift_in_mood_and_energy","fear_about_social_situations","fear_of_gaining_weight","fears_of_being_negatively_evaluated","flight_of_ideas","intrusion_symptoms","loss_of_interest_or_motivation","more_talktive","obsession","panic_fear","pessimism","poor_memory","sleep_disturbance","somatic_muscle","somatic_symptoms_others","somatic_symptoms_sensory","weight_and_appetite_change","Anger_Irritability"]

relevant_sents = []
postid = 0
sentid = 0

for post in df['sentences']:
    postid += 1
    for sent in post:
        sentid += 1
        meet_threshold = []
        if len(sent['sentence'].split()) >= 10:
            meet_threshold = [{'id':[postid,sentid],'sentence':sent['sentence']}]
            for symptom in symps:
                if sent['probabilities'][symptom] >= 0.3:
                    meet_threshold.append({symptom:'-'})
                    meet_threshold.append('annotation')
            if len(meet_threshold) >= 4:
                relevant_sents.append(meet_threshold)
        else:
            continue


print(len(relevant_sents))

df = pd.DataFrame(relevant_sents)
df.to_csv(f'../../../data/annotationData/{gender}annotationset.csv')



120
